In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
from auth import get_firebase_auth_headers

# Enter your username and password in a .env file
load_dotenv(".env")

# get authentication token
headers = get_firebase_auth_headers(
    os.environ['username'],
    os.environ['password']
)

### Get api information

In [13]:

base_url = "https://apiv2.raycast.nl"

url = f"{base_url}/"
response=requests.get(url)

if response.status_code != 200:
    print(response.text)
    
if response.status_code == 200:
    info_json=response.json()
    print(json.dumps(info_json, indent=4))


{
    "version": "0.0.7",
    "title": "Raycast API",
    "description": "Raycast API V2 - [Pythia Energy Intelligence](https://www.pythia-energy.nl/).",
    "contact": "info@pythia-energy.nl"
}


### Get credit information

In [14]:
# get the authorization headers

# endpoint to get the credit information
url = "https://api.raycast.nl/credits"

response=requests.get(url, headers=headers)

if response.status_code != 200:
    print(response.text)
    
if response.status_code == 200:
    credits_json=response.json()
    print(json.dumps(credits_json, indent=4))

{
    "credits": 32000,
    "name": "Thomas Stolp",
    "used": 402
}


### Get irradiance point

In [35]:
import pandas as pd

data = json.dumps({
    "val_datetime": "2024-11-19T13:45:00Z",
    "latitudes" : [52.924, 53.125, 52.101],
    "longitudes" : [4.785, 6.586, 5.177]
})

url = f"{base_url}/irradiance_point/"
response=requests.post(url, headers=headers, data=data)

if response.status_code != 200:
    print(response.text)

else:
    response_json = response.json()
    df = pd.DataFrame(response_json['data'])
    print(df.head())

           ref_datetime  lead_time  quantile       lon        lat      value
0  2024-11-19T13:45:00Z          1      0.01  4.782295  52.911646  13.933319
1  2024-11-19T13:45:00Z          1      0.05  4.782295  52.911646  26.276028
2  2024-11-19T13:45:00Z          1      0.10  4.782295  52.911646  29.709785
3  2024-11-19T13:45:00Z          1      0.20  4.782295  52.911646  39.574341
4  2024-11-19T13:45:00Z          1      0.30  4.782295  52.911646  48.726173


In [39]:
df

,ref_datetime,lead_time,quantile,lon,lat,value
0,2024-11-19T13:45:00Z,1,0.01,4.782295,52.911646,13.933319
1,2024-11-19T13:45:00Z,1,0.05,4.782295,52.911646,26.276028
2,2024-11-19T13:45:00Z,1,0.10,4.782295,52.911646,29.709785
3,2024-11-19T13:45:00Z,1,0.20,4.782295,52.911646,39.574341
4,2024-11-19T13:45:00Z,1,0.30,4.782295,52.911646,48.726173
...,...,...,...,...,...,...
541,2024-11-19T13:45:00Z,14,0.70,5.168876,52.090161,3.617974
542,2024-11-19T13:45:00Z,14,0.80,5.168876,52.090161,3.617975
543,2024-11-19T13:45:00Z,14,0.90,5.168876,52.090161,5.073140
544,2024-11-19T13:45:00Z,14,0.95,5.168876,52.090161,5.073141


### Get irradiance grid

In [36]:
import io
import xarray as xr

data = json.dumps({
    "datetime": "2024-11-19 13:45:00"
})

# endpoint to get the irradiance grid
url = f"{base_url}/irradiance/"

response=requests.post(url, headers=headers, data=data)

if response.status_code != 200:
    print(response.text)

# write content to a NetCDF file
if response.status_code == 200:
    with open('irradiance.nc', 'wb') as f:
        f.write(response.content)

# import as xarray dataset
if response.status_code == 200:
    file_obj = io.BytesIO(response.content)    
    ds = xr.open_dataset(file_obj)

In [37]:
ds

<xarray.Dataset> Size: 32MB
Dimensions:       (ref_datetime: 1, val_datetime: 14, lat: 152, lon: 146,
                   quantile: 13)
Coordinates:
  * ref_datetime  (ref_datetime) int64 8B 1732023900000000000
  * lon           (lon) float64 1kB 2.366 2.414 2.463 ... 9.276 9.325 9.373
  * lat           (lat) float64 1kB 55.33 55.28 55.23 ... 48.13 48.08 48.03
  * val_datetime  (val_datetime) int64 112B 1732024800000000000 ... 173203650...
  * quantile      (quantile) float64 104B 0.99 0.95 0.9 0.8 ... 0.1 0.05 0.01
Data variables:
    irradiance    (ref_datetime, val_datetime, lat, lon, quantile) float64 32MB ...
Attributes:
    title:       NETCDF File containing solar irradiance forecast
    contact:     info@pythia-energy.nl
    references:  https://pythia-energy.nl/